# INFO 4271 - Group Project

Issued: June 17, 2025

Due: July 21, 2025

Please submit a link to your code repository (with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [ ]:
%load_ext autoreload
%autoreload 2

from project import Crawler
import cProfile

start_urls = []
with open('seed.txt', 'r') as file:
    for line in file: start_urls.append(line.strip())
start_urls
crawler = Crawler(start_urls, max_workers=200, use_proxies=False, auto_resume=False, path="data")
# snakeviz profiler.prof
# cProfile.run('crawler.run()', 'data/profiler.prof')
crawler.run()

In [ ]:
crawler.get_crawling_stats()

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file (see `queries.txt` for an example) will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
%load_ext autoreload
%autoreload 2

from project import HTML_FILE
from tqdm import tqdm
import os
import json

def load_data(path: str = "data"):
    html_path = os.path.join(path, HTML_FILE)
    if not os.path.exists(html_path):
        raise FileNotFoundError(f"HTML file not found at {html_path}")

    docs = {}
    with open(html_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in tqdm(lines, "Line"): docs.update(json.loads(line.strip()))
    return docs

docs = load_data(path="data")

In [ ]:
from bs4 import BeautifulSoup

def preprocess_html(html: str, seperator: str = ' ') -> str:
    soup = BeautifulSoup(html, 'lxml')
    text = soup.get_text(separator=seperator, strip=True)
    return text.strip()

In [ ]:
from project import SiglipStyleModel, ColSentenceModel
import torch

def generate_embeddings(model: SiglipStyleModel | ColSentenceModel, data_dict: dict, batch_size: int = 1) -> dict:
    embedding_dict = {}
    items = list(data_dict.items())
    for i in tqdm(range(0, len(data_dict), batch_size), "Embedding"):
        htmls = [preprocess_html(item[1], ". ")[:3000] for item in items[i:i + batch_size]]
        try:
            embeddings = model.embed(htmls).detach().cpu()
            for j, embedding in enumerate(embeddings):
                embedding_dict[embedding] = items[i + j][0]
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"Error embedding: {e}")
    return embedding_dict

model = ColSentenceModel().load(r"project\retriever\model_uploads\bmini_ColSent_b128_marco_v1.safetensors")
# model = SiglipStyleModel().load(r"project/retriever/model_uploads/bert-mini_b32_marco_v1.safetensors")
embeddings = generate_embeddings(model, docs)

In [ ]:
import torch

def save_embeddings(embeddings: dict, path: str = "data/embeddings.pkl"):
    torch.save(embeddings, path)

save_embeddings(embeddings, path="data/embeddings.pkl")

In [ ]:
import torch

def load_embeddings(path: str = "data/embeddings.pkl") -> dict:
    return torch.load(path)

embedding_dict = load_embeddings(path="data/embeddings.pkl")

In [ ]:
import numpy as np

def retrieve(model: SiglipStyleModel | ColSentenceModel, query: str, index: dict[torch.Tensor, str]):
    similarities = []
    query_embedding = model.embed(query)
    for embedding, _ in tqdm(list(index.items()), "Similarities"):
        similarity = model.resolve(query_embedding, embedding.cuda()).squeeze()
        similarities.append(similarity.detach().cpu())
    vals = np.array(list(zip(index.values(), similarities)))
    return vals[np.argsort(similarities)[::-1]]

model = ColSentenceModel().load(r"project\retriever\model_uploads\bmini_ColSent_b128_marco_v1.safetensors")
# model = SiglipStyleModel().load(r"project/retriever/model_uploads/bert-mini_b32_marco_v1.safetensors")
res = retrieve(model, "food and drinks", embedding_dict)
res[:30]

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [ ]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 22nd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
